In [ ]:
from sigmf_pytorch_dataset import SigMFDataset

In [ ]:
dataset = SigMFDataset( root=".")

In [ ]:
from torchsig.utils.visualize import IQVisualizer, SpectrogramVisualizer, two_channel_to_complex
from torchsig.utils.dataset import SignalDataset
from torchsig.datasets.sig53 import Sig53
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from typing import List
from tqdm import tqdm
import numpy as np

In [ ]:
data, label = dataset[0]
print("Data shape: {}".format(data.shape))
print("Label shape: {}".format(label))

In [ ]:
data_loader = DataLoader(
    dataset=dataset,
    batch_size=4,
    shuffle=True,
)

labels = []
for _, label in dataset:
    labels.append(label)

print("Unique labels: {}".format(np.unique(labels)))

In [ ]:
data_loader = DataLoader(
    dataset=dataset,
    batch_size=100,
    shuffle=True,
)

visualizer = IQVisualizer(
    data_loader=data_loader
)

for figure in iter(visualizer):
    figure.set_size_inches(16, 16)
    plt.show()
    break

# SigMF based Model Training

In [ ]:
from torchsig.models.iq_models.efficientnet.efficientnet import efficientnet_b4
from pytorch_lightning.callbacks import ModelCheckpoint
from torchsig.utils.cm_plotter import plot_confusion_matrix
from pytorch_lightning import LightningModule, Trainer
from sklearn.metrics import classification_report
from torchsig.datasets.sig53 import Sig53
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F
import torchsig.transforms as ST
import numpy as np
import torchsig
import torch
import os
from sigmf_db_dataset import SigMFDB
from sigmf_pytorch_dataset import SigMFDataset

### Load the SigMF File dataset
and generate the class list

In [ ]:
transform = ST.Compose([
    ST.RandomPhaseShift(phase_offset=(-1, 1)),
    ST.Normalize(norm=np.inf),
    ST.ComplexTo2D(),
])
class_list = ['signal', 'mini2_video']
dataset = SigMFDataset( root=".",
                       transform=transform,
                       class_list=class_list,
)
labels = []
for _, label in dataset:
    labels.append(label)
#class_list=np.unique(labels).tolist()
print("Unique labels: {}".format(np.unique(labels)))

### SigMF DB Data Loaders
The number of workers needs to be set to 0 when there is not a GPU. Otherwise the dataloader tries to pickle the environment variables: https://github.com/acids-ircam/RAVE/issues/10#issuecomment-1002708207

In [ ]:

# Specify Sig53 Options

root = "sigmf/"
train = False
impaired = False
class_list = ['signal', 'mini2_video']
print("Class List: {}".format(class_list))
transform = ST.Compose([
    ST.RandomPhaseShift(phase_offset=(-1, 1)),
    ST.Normalize(norm=np.inf),
    ST.ComplexTo2D(),
])
target_transform = ST.DescToClassIndex(class_list=class_list)

sig53_clean_train = SigMFDB(
    root=root, 
    transform=transform,
    target_transform=target_transform,
    use_signal_data=True,
)

# Retrieve a sample and print out information to verify
print(len(sig53_clean_train))
idx = np.random.randint(len(sig53_clean_train))
data, label = sig53_clean_train[idx]
print("Dataset length: {}".format(len(sig53_clean_train)))
print("Data shape: {}".format(data.shape))
print("Data Object: {}".format(data))
print("Label Index: {}".format(label))


train_dataloader = DataLoader(
    sig53_clean_train ,
    batch_size=16,
    num_workers=0,
    shuffle=True,
    drop_last=True,
)
val_dataloader = DataLoader(
    sig53_clean_train ,
    batch_size=16,
    num_workers=0,
    shuffle=False,
    drop_last=True,
)

### SigMF File Dataloaders
This is working

In [ ]:




train_dataloader = DataLoader(
    dataset=dataset, #sig53_clean_train ,
    batch_size=16,
    num_workers=8,
    shuffle=True,
    drop_last=True,
)
val_dataloader = DataLoader(
    dataset=dataset, #sig53_clean_train ,
    batch_size=16,
    num_workers=8,
    shuffle=False,
    drop_last=True,
)

In [ ]:
model = efficientnet_b4(
    pretrained=True,
    path="efficientnet_b4.pt",
)

device = torch.device('cpu') #('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
class ExampleNetwork(LightningModule):
    def __init__(self, model, data_loader, val_data_loader):
        super(ExampleNetwork, self).__init__()
        self.mdl = model
        self.data_loader = data_loader
        self.val_data_loader = val_data_loader

        # Hyperparameters
        self.lr = 0.001
        self.batch_size = data_loader.batch_size

    def forward(self, x):
        return self.mdl(x)

    def predict(self, x):
        with torch.no_grad():
            out = self.forward(x)
        return out

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return self.data_loader

    def training_step(self, batch, batch_nb):
        x, y = batch
        print(x.shape)
        print(y.shape)
        y = torch.squeeze(y.to(torch.int64))
        loss = F.cross_entropy(self(x.float()), y)
        return {"loss": loss}

    def val_dataloader(self):
        return self.val_data_loader

    def validation_step(self, batch, batch_nb):
        x, y = batch
        y = torch.squeeze(y.to(torch.int64))
        val_loss = F.cross_entropy(self(x.float()), y)
        self.log("val_loss", val_loss, prog_bar=True)
        return {"val_loss": val_loss}

example_model = ExampleNetwork(model, train_dataloader, val_dataloader)

In [ ]:
# Setup checkpoint callbacks
checkpoint_filename = "{}/checkpoints/checkpoint".format(os.getcwd())
checkpoint_callback = ModelCheckpoint(
    filename=checkpoint_filename,
    save_top_k=True,
    monitor="val_loss",
    mode="min",
)
print("Doing stuff")
# Create and fit trainer
epochs = 25
trainer = Trainer(
    max_epochs=epochs, callbacks=checkpoint_callback, accelerator="cpu", devices=1
)
trainer.fit(example_model)